# **Selenium 사용하지 않고 해보기**

In [5]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import time
import logging
import pandas as pd

# 로그 출력을 위해서 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## **1. url crawling**

In [12]:
import requests
from bs4 import BeautifulSoup
import time
import logging
import random

# 로그 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

search_query = '기독교'  # 검색어 설정
encoded_query = requests.utils.quote(search_query)

news_urls = set()  # ✅ 중복 제거를 위해 리스트 대신 `set()` 사용

# 네이버 뉴스 검색 URL 패턴
base_url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query='

# ✅ 다양한 User-Agent 리스트 (랜덤 사용)
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.119 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36",
]

def get_news_urls(page):
    """각 페이지에서 네이버 뉴스 URL을 수집하는 함수"""
    url = f'{base_url}{encoded_query}&start={page*10+1}'
    headers = {
        'User-Agent': random.choice(user_agents)  # ✅ User-Agent를 랜덤으로 변경
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # ✅ 더 정확한 뉴스 제목 선택자 사용 (`.news_tit`으로 변경)
        links = soup.select('a.news_tit')

        if not links:
            logging.warning(f'페이지 {page}: 뉴스 기사 없음 (차단 가능성 있음)')
            return False  # 더 이상 링크가 없으면 False 반환

        for link in links:
            news_url = link['href']

            # ✅ 네이버 뉴스 URL인지 확인하고 중복 제거 (`set()` 사용)
            if 'news.naver.com' in news_url:
                news_urls.add(news_url)

        logging.info(f'페이지 {page}: {len(links)}개 기사 중 {len(news_urls)}개 수집됨')
        return True

    except requests.exceptions.RequestException as e:
        logging.error(f'페이지 {page} 요청 실패: {e}')
        return False

# ✅ 네이버 뉴스 1000개 이상 수집
page = 0
max_pages = 200  # 최대 200페이지 크롤링
while len(news_urls) < 1000 and page < max_pages:
    if not get_news_urls(page):
        break
    page += 1
    time.sleep(random.uniform(2, 5))  # ✅ 요청 간 딜레이를 랜덤하게 조정 (차단 방지)

# ✅ 최종 결과 출력
logging.info(f'총 {len(news_urls)}개의 네이버 뉴스 URL을 수집했습니다.')

# ✅ 수집된 뉴스 URL을 파일로 저장
txt_filename = f"{search_query}_naver_news_urls.txt"
with open(txt_filename, 'w', encoding='utf-8') as f:
    for url in news_urls:
        f.write(url + '\n')

logging.info(f"URL 리스트가 {txt_filename}파일에 저장되었습니다.")


2025-02-10 19:28:43,017 - INFO - 페이지 0: 10개 기사 중 0개 수집됨
2025-02-10 19:28:45,851 - INFO - 페이지 1: 10개 기사 중 0개 수집됨


KeyboardInterrupt: 

## **2. 본문 crawling**

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import logging
import pandas as pd
import html
import gzip
import brotli

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 저장된 뉴스 URL 파일명
# txt_filename = f"{search_query}_urls_{len(news_urls)}.txt"
txt_filename = f"{search_query}_urls_1206.txt"

# 뉴스 URL 불러오기 (100개만 선택)
with open(txt_filename, 'r', encoding='utf-8') as f:
    news_urls = [line.strip() for line in f.readlines()][:100]

logging.info(f"{len(news_urls)}개의 뉴스 URL을 불러왔습니다. (테스트 모드)")

# 기사 제목과 본문 저장할 리스트
news_data = []
headers = {'User-Agent': 'Mozilla/5.0'}

# 뉴스 본문 크롤링
for idx, url in enumerate(news_urls):
    try:
        response = requests.get(url, headers=headers)

        # 🔹 압축 해제 시도 전에 gzip/brotli 여부 확인 후 실행
        encoding = response.headers.get('Content-Encoding', '')

        if encoding == 'br':
            try:
                response._content = brotli.decompress(response.content)
            except Exception as e:
                logging.warning(f"URL {idx + 1}/{len(news_urls)}: Brotli 압축 해제 실패 - {e}")

        elif encoding == 'gzip':
            try:
                response._content = gzip.decompress(response.content)
            except Exception as e:
                logging.warning(f"URL {idx + 1}/{len(news_urls)}: Gzip 압축 해제 실패 - {e}")

        # 🔹 인코딩 자동 감지 (utf-8 기본 적용)
        response.encoding = response.apparent_encoding if 'charset' in response.headers.get('content-type', '').lower() else 'utf-8'

        if response.status_code != 200:
            logging.warning(f"URL {idx + 1}/{len(news_urls)} 요청 실패: {response.status_code}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # 기사 제목 가져오기
        title = soup.select_one('h2#title_area') or soup.select_one('h1')
        title_text = title.get_text(strip=True) if title else "제목 없음"

        # 기사 본문 가져오기
        content = soup.select_one('#dic_area') or soup.select_one('article') or soup.select_one('.news_end')
        content_text = content.get_text(strip=True) if content else "본문 없음"

        news_data.append([url, title_text, content_text])
        logging.info(f"{idx + 1}/{len(news_urls)}: 기사 크롤링 완료 - {title_text[:30]}...")

        time.sleep(1)  # 요청 간 딜레이

    except Exception as e:
        logging.error(f"URL {idx + 1}/{len(news_urls)} 크롤링 중 오류 발생: {e}")
        continue

# 크롤링 결과를 CSV 파일로 저장
df = pd.DataFrame(news_data, columns=['URL', '제목', '본문'])
csv_filename = f"{search_query}_test_100_news_contents.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

logging.info(f"크롤링 완료! 100개 기사 데이터가 '{csv_filename}' 파일에 저장되었습니다.")


2025-02-10 18:37:41,919 - INFO - 100개의 뉴스 URL을 불러왔습니다. (테스트 모드)
2025-02-10 18:37:42,007 - WARNING - URL 1/100: Gzip 압축 해제 실패 - Not a gzipped file (b'<!')
2025-02-10 18:37:42,044 - INFO - 1/100: 기사 크롤링 완료 - [한인사회 소식[ 시카고평통-기독교방송국-명계웅 추모식...
2025-02-10 18:37:43,474 - WARNING - URL 2/100: Gzip 압축 해제 실패 - Not a gzipped file (b'<!')
2025-02-10 18:37:43,949 - INFO - 2/100: 기사 크롤링 완료 - '제3공화국'·'사랑과 야망' 감초역할, 최병학 별세 ...
2025-02-10 18:37:46,202 - WARNING - URL 3/100: Gzip 압축 해제 실패 - Not a gzipped file (b'\xef\xbb')
2025-02-10 18:37:46,228 - INFO - 3/100: 기사 크롤링 완료 - NSP통신...
2025-02-10 18:37:47,446 - WARNING - URL 4/100: Gzip 압축 해제 실패 - Not a gzipped file (b'<!')
2025-02-10 18:37:47,550 - INFO - 4/100: 기사 크롤링 완료 - 김종혁 한교총 대표회장 예방한 권영세 비대위원장...
2025-02-10 18:37:48,597 - WARNING - URL 5/100: Gzip 압축 해제 실패 - Not a gzipped file (b'<!')
2025-02-10 18:37:48,691 - INFO - 5/100: 기사 크롤링 완료 - ...
2025-02-10 18:37:50,485 - WARNING - URL 6/100: Gzip 압축 해제 실패 - Not a gzipped file (b'\r\n')
2025-02-10 18:37:

In [ ]:
df['본문']

0       To write comments, please log in to one of the...
1                                                   본문 없음
2                                                   본문 없음
3       [서울=뉴시스] 추상철 기자 = 권영세 국민의힘 비상대책위원장이 13일 오후 서울 ...
4         기자명윤여군입력 2024.07.29 18:47수정 2024.07.29 19:12댓글1
                              ...                        
1193    오늘의뉴스뉴스데스크뉴스투데이[한걸음 더]취재가시작되자광주MBC단독공항이전이슈5.18...
1194                                                본문 없음
1195    관련 키워드메르켈독일이민극우기민당AfD이창규 기자"아내와 두딸 테러에 몰살"…하마스...
1196                         기자명허현정 기자입력 2025.02.03 16:07
1197    시사전체기사정치 견해 다르다고 악마화? “이렇게 해봅시다”입력:2025-02-02 ...
Name: 본문, Length: 1198, dtype: object

- selenium으로 content, title 추출

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

df = pd.read_csv(output_filename )
# 1000개까지만 url 가져오기
df = df.head(1000)

# Chrome WebDriver 서비스 생성
service = Service("C:\\chromedriver-win64\\chromedriver.exe")
options = Options()
# options.add_argument("--headless")  # 필요 시 헤드리스 모드 사용
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(3)

# 새로운 데이터를 저장할 리스트
data = []

# 스크래핑 시작 시간 기록
start_time = time.time()

# URL 열에서 URL을 행마다 뽑아서 스크래핑 작업 수행
for index, row in df.iterrows():
    link = row['URL']
    logging.info(f'{index + 1}번째 URL 스크랩 중: {link}')
    time.sleep(2)
    driver.get(link)
    
    try:
        # 제목 추출하기
        title_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'media_end_head_headline'))
        )
        title = title_element.text
        logging.info(f'{index + 1}번째 URL 제목 추출 완료')
        
        # 본문 추출하기
        body_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'newsct_article'))
        )
        body = body_element.text.replace('\n', '')
        logging.info(f'{index + 1}번째 URL 본문 추출 완료')
        
        # 데이터를 리스트에 추가
        data.append({
            'link': link,
            'title': title,
            'contnet': body
        })
    except Exception as e: #except 설정안하면 오류나면 코드가 멈춤
        logging.error(f"{index + 1}번째 URL 스크래핑 중 오류 발생: {link}, 오류: {e}")

driver.quit()

# 스크래핑 종료 시간 기록
end_time = time.time()
elapsed_time = end_time - start_time
logging.info(f"Total elapsed time: {elapsed_time:.2f} seconds")

# DataFrame으로 변환
result_df = pd.DataFrame(data)

# 결과 출력
print(result_df)

###키워드변경
output_filename = f'Selenium_news_{search_query}.csv'

# 결과를 CSV 파일로 저장
result_df.to_csv(output_filename, index=False)


2024-06-02 19:46:00,698 - INFO - 1번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/421/0007533064?sid=102
2024-06-02 19:46:03,517 - INFO - 1번째 URL 제목 추출 완료
2024-06-02 19:46:03,647 - INFO - 1번째 URL 본문 추출 완료
2024-06-02 19:46:03,649 - INFO - 2번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/005/0001698400?sid=103
2024-06-02 19:46:06,129 - INFO - 2번째 URL 제목 추출 완료
2024-06-02 19:46:06,203 - INFO - 2번째 URL 본문 추출 완료
2024-06-02 19:46:06,204 - INFO - 3번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/422/0000658212?sid=102
2024-06-02 19:46:08,737 - INFO - 3번째 URL 제목 추출 완료
2024-06-02 19:46:08,774 - INFO - 3번째 URL 본문 추출 완료
2024-06-02 19:46:08,775 - INFO - 4번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/087/0001018030?sid=102
2024-06-02 19:46:11,167 - INFO - 4번째 URL 제목 추출 완료
2024-06-02 19:46:11,194 - INFO - 4번째 URL 본문 추출 완료
2024-06-02 19:46:11,194 - INFO - 5번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/003/0012283299?sid=104
2024-06-02 19:46:13,515 - INFO - 5번째 URL 제목 추출 완료


                                                  link  \
0    https://n.news.naver.com/mnews/article/421/000...   
1    https://n.news.naver.com/mnews/article/005/000...   
2    https://n.news.naver.com/mnews/article/422/000...   
3    https://n.news.naver.com/mnews/article/087/000...   
4    https://n.news.naver.com/mnews/article/003/001...   
..                                                 ...   
995  https://n.news.naver.com/mnews/article/079/000...   
996  https://n.news.naver.com/mnews/article/003/001...   
997  https://n.news.naver.com/mnews/article/081/000...   
998  https://n.news.naver.com/mnews/article/005/000...   
999  https://n.news.naver.com/mnews/article/032/000...   

                                                title  \
0                "세계기독교 성지 건설하는데 자금부족"…1억 챙긴 80대 '실형'   
1                           [새로 나온 책] 당신의 관계는 안녕하십니까?   
2                     현직 경찰, JMS 정명석 증거인멸 연루 의혹…감사 착수   
3                          [포토뉴스]2024년 춘천기독교연합회 신년교례회   
4                 

In [ ]:
result_df

NameError: name 'result_df' is not defined